In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기
가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 한다. torch.cuda 또는 torch.backends.mps가 사용가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용한다.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


#### 클래스 정의하기 
신경망 모델을 nn.Module의 하위 클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다. nn.Modele을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


### Model Layer
FashionMNIST 모델의 레이어를 분석해본다.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
nn.Flatten Layer을 초기화하여 각 28 * 28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열로 변환한다. (dim = 0의 미니배치 차원은 유지된다)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear
Linear Layer는 가중치(weight)와 편향(bias)를 사용하여 입력에 선형 변환(Linear transformation)을 적용하는 모듈이다.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2002,  0.3685,  0.8839,  0.2427,  0.0777,  0.5713, -0.2637, -0.1987,
         -0.0698,  0.0473,  0.1099,  0.4739,  0.1207,  0.1972, -0.3698, -0.0733,
         -0.3414, -0.2336, -0.1807,  0.1402],
        [-0.3390,  0.5822,  0.7392,  0.2595,  0.0187,  0.6626, -0.1510, -0.5012,
         -0.1165, -0.1157, -0.1946,  0.5475,  0.2707,  0.2504, -0.1932, -0.1731,
         -0.4110, -0.3225, -0.5769, -0.0456],
        [-0.1090,  0.3625,  0.5586, -0.2882, -0.1520,  0.4636, -0.0283, -0.3950,
         -0.1728,  0.2591,  0.1435,  0.6109,  0.3145,  0.3518,  0.3801, -0.3324,
          0.0113, -0.2516, -0.3152, -0.0069]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3685, 0.8839, 0.2427, 0.0777, 0.5713, 0.0000, 0.0000, 0.0000,
         0.0473, 0.1099, 0.4739, 0.1207, 0.1972, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1402],
        [0.0000, 0.5822, 0.7392, 0.2595, 0.0187, 0.6626, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.5475, 0.2707, 0.2504, 0.00

#### nn.Sequential 
nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너를 사용하여 아래의 seq_moduels와 같은 신경망을 빠르게 만들 수 있다.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

#### nn.Softmax
신경망의 마지막 레이어 nn.Softmax 모듈은 [-inf, inf]범위의 값을 logits 변환한다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정된다. dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [11]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

#### 모델 매개변수
신경망 내부의 많은 계층들은 매개변수화 된다. 즉, 학습 중에 최적화되는 가중치와 편향을 찾아야 한다. nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(Tracking)되며, 모델의 parameters()및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다. 아래 코드는 각 매개변수들을 순회하며, 매개변수의 크기와 값을 출력한다.

In [12]:
print(f"Model structure {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}\n")

Model structure NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0235, -0.0322, -0.0082,  ..., -0.0337,  0.0022, -0.0105],
        [ 0.0212, -0.0299, -0.0187,  ..., -0.0282,  0.0197,  0.0314]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0312, -0.0236], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0336,  0.0206, -0.0368,  ..., -0.0358, -0.0385, -0.0051],
        [-0.0362,  0.0120, -0.0164,  ..., -0.0347,  0.0195,  0.0180]],
       device='cuda:0', grad_fn=<Slice